<a href="https://colab.research.google.com/github/Divyanshu-ISM/Machine-Learning-Deep-Learning/blob/main/Model_Selection_%26_HyperParameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Selection & HyperParameter Tuning

> ### 1. GridSearchCV
> ### 2. Randomized Search CV

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

In [7]:
#Optional
df = pd.DataFrame(data=X,columns=iris.feature_names)
df['Species'] = y
# df.head()
namer = lambda x : iris.target_names[x]
df['Species'] = df['Species'].apply(namer)

In [8]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [17]:
#Instantiate a Grid-Search Object
grid = GridSearchCV(SVC(gamma='auto'),
                    {'C':np.linspace(1,30),
                     'kernel':['rbf','linear']},
                    cv=5,return_train_score=False)

#Fit the Grid-Search object to data. 
grid.fit(X,y)
grid.best_params_

{'C': 1.5918367346938775, 'kernel': 'rbf'}

In [21]:
grid_results = pd.DataFrame(grid.cv_results_)
grid_results = grid_results[['param_C','param_kernel','mean_test_score']]
grid_results.head()

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,1.59184,rbf,0.986667
3,1.59184,linear,0.986667
4,2.18367,rbf,0.980000


In [22]:
max(grid_results['mean_test_score'])

0.9866666666666667

# Selecting the best Model out of all the prospects 

In [23]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [24]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
modelDF = pd.DataFrame(scores,columns=['model','best_score','best_params'])
modelDF

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 1}
2,logistic_regression,0.966667,{'C': 5}
